This note is used for updating cell calling in Tony's perturb pipeline 

In [10]:
import pandas as pd
import sys
from IPython.display import display
pd.set_option('display.max_colwidth', None)
print(sys.executable)

import argparse
import subprocess
import sys
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import scanpy as sc
import os
from types import SimpleNamespace 

/oak/stanford/groups/engreitz/Users/shshanhe/tools/miniforge3/envs/torch-cNMF/bin/python


In [9]:
!which python
!which Rscript

/oak/stanford/groups/engreitz/Users/shshanhe/tools/miniforge3/envs/torch-cNMF/bin/python
which: no Rscript in (/oak/stanford/groups/engreitz/Users/shshanhe/tools/miniforge3/envs/torch-cNMF/bin:/home/users/shshanhe/miniconda3/condabin:/share/software/user/open/code-server/4.93.1/lib/node_modules/code-server/lib/vscode/bin/remote-cli:/share/software/user/srcc/bin:/share/software/user/open/nodejs/25.3.0/bin:/share/software/user/open/gcc/14.2.0/bin:/share/software/user/open/code-server/4.93.1/bin:/share/software/user/open/git/2.45.1/bin:/share/software/user/open/expat/2.2.3/bin:/share/software/user/open/curl/8.4.0/bin:/share/software/user/open/openssl/3.0.7/bin:/share/software/user/srcc/bin:/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/users/shshanhe/.local/bin:/home/users/shshanhe/bin:/home/users/shshanhe/.local/bin:/home/users/shshanhe/bin)


# function loading

In [11]:
import os
import argparse
import scanpy as sc
import pandas as pd
import numpy as np
import scipy
from scipy import io
import gc
import sys
import yaml
import re
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from types import SimpleNamespace 

utils_path = "/oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/scripts"
if utils_path not in sys.path:
    sys.path.append(utils_path)
    print(f"✅ add path: {utils_path}")

try:
    from pipeline_utils import (
        log_print, 
        add_comprehensive_gene_annotations,
        add_mitochondrial_metrics,
        map_cells_to_samples_with_plate,
        filter_guides_by_reference,
        align_datasets_by_cells,
        add_guide_data,
        get_guide_gex_pairings
    )
    from sublibrary_annotation import (
        log_print, 
        filter_and_prepare_gex, 
        filter_and_prepare_guide, 
        add_guide_data, 
        add_guide_statistics, 
        add_comprehensive_gene_annotations, 
        add_mitochondrial_metrics, 
        map_cells_to_samples_with_plate, 
        save_mtx_files
    )
    print("✅ pipeline_utils")
    

except ImportError as e:
    print(f"❌: {e}")


✅ add path: /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/scripts
✅ pipeline_utils


# test main code

In [12]:
args = SimpleNamespace(
    gex_kb_dir     = "/scratch/users/shshanhe/eRZ58_default_251225/eRZ58_20A_analysis/eRZ58:eRZ58_GEX20A/kb_all_main_raw/", 
    guide_kb_dir   = "/scratch/users/shshanhe/eRZ58_default_251225/eRZ58_20A_analysis/eRZ58:eRZ58_gRNA20A/kb_guide_main_raw/", 
    output_dir     = "/oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX20A_UMI_1",
    cutoff         = 1, # config['sublibrary_filtering']['guide_assignment_cutoff'],
    config         = "/oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/config.eRZ59.yaml",
    sample_id      = "eRZ58:eRZ58_GEX20A", 
    guide_sample_id= "eRZ58:eRZ58_gRNA20A", 
    skip_filtering = True,   
    source         = "all",   # main, undetermined, or all
    processing     = "raw",   # raw, trimmed, recovered, merged
    ncores         = 4
)

os.makedirs(args.output_dir, exist_ok=True)

print(f"Processing GEX: {args.sample_id}")


Processing GEX: eRZ58:eRZ58_GEX20A


In [14]:
# Cell 3: Main Execution Logic

# Start pipeline logic
log_print("\n" + "=" * 80)
log_print("🚀 STARTING SUBLIBRARY FILTERING AND ANNOTATION PIPELINE (NOTEBOOK MODE)")

# Extract pool from sample_id
if ':' not in args.sample_id:
    raise ValueError(f"Invalid sample_id format: {args.sample_id}. Expected 'pool:sample'")
pool = args.sample_id.split(':')[0]

log_print(f"📊 Processing: Pool {pool}, GEX {args.sample_id}, Guide {args.guide_sample_id}")

# Load config
with open(args.config, 'r') as f:
    config = yaml.safe_load(f)

# Sample info file
sample_info_file = config['sample_info_file']

# Cell filtering parameters
# 如果 args.skip_filtering 为 True，这个值实际上不会被用于过滤，但可能用于日志记录
min_umi_filter = config['sublibrary_filtering']['min_umi_filter'] 
if args.skip_filtering:
    log_print(f"⚠️  NOTE: args.skip_filtering is True. Original config min_umi ({min_umi_filter}) will be IGNORED.")
    # 实际上为了保证下游兼容性，我们可能只需确保 filter_and_prepare_gex 内部尊重 skip_filtering 标志

# Gene annotation reference files
ribosomal_genes_path = config['input_paths']['ribosomal_genes']
gene_database_path = config['input_paths']['gene_database']
cell_cycle_genes_path = config['input_paths']['cell_cycle_genes']
parsebio_barcodes_path = config['input_paths']['parsebio_barcodes']
plate_maps_file = config.get('plate_maps_file')

# Load barcode file once
parsebio_barcodes = pd.read_csv(parsebio_barcodes_path)

# Load sample info
log_print("📋 Loading sample information...")
sample_df = pd.read_excel(sample_info_file)
sample_row = sample_df[sample_df['sample_id'] == args.sample_id]

if sample_row.empty:
    raise ValueError(f"Sample {args.sample_id} not found in {sample_info_file}")

# Extract expected cells
expected_cells = int(sample_row.iloc[0]['expected_cells'])
plate_name = sample_row.iloc[0].get('sample_to_well_mapping')

log_print(f"   Sample: {args.sample_id}")
log_print(f"   Expected cells: {expected_cells}")

# --- 1. LOAD AND FILTER GEX DATA ---
log_print("\n📥 1. LOADING GEX DATA...")
# 注意：这里我们强制读取 unfiltered 数据
gex_h5ad_path = Path(args.gex_kb_dir) / "counts_unfiltered_modified" / "adata.h5ad"
log_print(f"   Loading from: {gex_h5ad_path}")

adata_gex = sc.read_h5ad(gex_h5ad_path)
log_print(f"   Loaded raw matrix: {adata_gex.shape[0]} cells x {adata_gex.shape[1]} genes")

# 关键：调用 filter 函数
# 确保你的 filter_and_prepare_gex 函数内部有处理 skip_filtering 的逻辑
adata_gex = filter_and_prepare_gex(
    adata_gex, 
    args.sample_id, 
    expected_cells, 
    args.skip_filtering, # 这里传入 True，保留背景
    min_umi_filter
)
gc.collect()

# --- 2. LOAD AND FILTER GUIDE DATA ---
log_print("\n📥 2. LOADING GUIDE DATA...")
guide_h5ad_path = Path(args.guide_kb_dir) / "counts_unfiltered_modified" / "adata.h5ad"
adata_guide = sc.read_h5ad(guide_h5ad_path)

# Filter guide to match GEX barcodes (even if GEX is unfiltered, we match indices)
adata_guide = filter_and_prepare_guide(
    adata_guide, 
    args.guide_sample_id, 
    adata_gex.obs.index.tolist(), 
    args.skip_filtering
)
gc.collect()

# --- 3. COMBINE GEX AND GUIDE DATA ---
log_print("\n🔗 3. COMBINING DATA...")
adata = add_guide_data(adata_gex, adata_guide)
add_guide_statistics(adata, cutoff=args.cutoff)

del adata_gex, adata_guide
gc.collect()

# --- 4. ANNOTATION & QC ---
log_print("\n🏷️  4. ANNOTATING...")
add_comprehensive_gene_annotations(adata, ribosomal_genes_path, gene_database_path, cell_cycle_genes_path)
add_mitochondrial_metrics(adata)

adata.obs['pool'] = pool
adata.obs['sample_id'] = args.sample_id

# Plate mapping
if not pd.isna(plate_name) and plate_maps_file:
    log_print(f"   Mapping plate: {plate_name}")
    map_cells_to_samples_with_plate(adata, plate_name, plate_maps_file, parsebio_barcodes)

# --- 5. SAVE OUTPUTS FOR CELL CALLING ---
log_print("\n💾 5. SAVING OUTPUTS...")
output_path = Path(args.output_dir)

# Save h5ad
adata.write_h5ad(output_path / "adata_unfiltered_annotated.h5ad")

# Save MTX files (Crucial for Cell Calling tools like dropletutils or split-pipe)
# 这一步生成 matrix.mtx, barcodes.tsv, features.tsv
save_mtx_files(adata, output_path, sample_type="gex")

log_print(f"\n✅ Ready for Cell Calling Benchmark!")
log_print(f"   Output saved to: {output_path}")

2026-01-21 05:55:30,429 - INFO - 
2026-01-21 05:55:30,430 - INFO - 🚀 STARTING SUBLIBRARY FILTERING AND ANNOTATION PIPELINE (NOTEBOOK MODE)
2026-01-21 05:55:30,430 - INFO - 📊 Processing: Pool eRZ58, GEX eRZ58:eRZ58_GEX20A, Guide eRZ58:eRZ58_gRNA20A
2026-01-21 05:55:30,447 - INFO - ⚠️  NOTE: args.skip_filtering is True. Original config min_umi (1) will be IGNORED.
2026-01-21 05:55:30,450 - INFO - 📋 Loading sample information...
2026-01-21 05:55:30,543 - INFO -    Sample: eRZ58:eRZ58_GEX20A
2026-01-21 05:55:30,543 - INFO -    Expected cells: 5000
2026-01-21 05:55:30,544 - INFO - 
📥 1. LOADING GEX DATA...
2026-01-21 05:55:30,545 - INFO -    Loading from: /scratch/users/shshanhe/eRZ58_default_251225/eRZ58_20A_analysis/eRZ58:eRZ58_GEX20A/kb_all_main_raw/counts_unfiltered_modified/adata.h5ad
2026-01-21 05:55:35,527 - INFO -    Loaded raw matrix: 400356 cells x 34183 genes
2026-01-21 05:55:35,785 - INFO -    Summed mature + nascent + ambiguous layers
2026-01-21 05:55:35,786 - INFO -    adata.X

In [44]:
adata
#adata.obsm['guide_counts'] # Guide data stored in obsm['guide_counts'] (zeros for cells without guide data)

AnnData object with n_obs × n_vars = 453765 × 34183
    layers: 'ambiguous', 'mature', 'nascent'

In [48]:
import scipy.sparse as sp

adata = sc.read_h5ad("/scratch/users/shshanhe/eRZ58_default_251225/eRZ58_20A_analysis/eRZ58:eRZ58_GEX20A/kb_all_main_raw/counts_filtered/adata.h5ad")

G = adata.obsm["guide_assignment"]
assert sp.issparse(G), "guide_assignment 不是 sparse matrix？请检查类型"

# 每个细胞被分配到的 guide 数（因为是 binary，所以 sum 就是 count）
n_guides_per_cell = np.asarray(G.sum(axis=1)).ravel()

n_cells_total = adata.n_obs
n_cells_with_guide = int((n_guides_per_cell > 0).sum())
n_cells_multi_guide = int((n_guides_per_cell > 1).sum())

print(f"Total cells: {n_cells_total}")
print(f"Cells with >=1 gRNA assigned: {n_cells_with_guide} ({n_cells_with_guide/n_cells_total:.2%})")
print(f"Cells with >1 gRNA assigned (multi-guide): {n_cells_multi_guide} ({n_cells_multi_guide/n_cells_total:.2%})")


Total cells: 4160
Cells with >=1 gRNA assigned: 3527 (84.78%)
Cells with >1 gRNA assigned (multi-guide): 2582 (62.07%)


# output

In [39]:
2026-01-20 10:13:40,963 - INFO - 
================================================================================
2026-01-20 10:13:40,964 - INFO - 🚀 STARTING SUBLIBRARY FILTERING AND ANNOTATION PIPELINE (NOTEBOOK MODE)
2026-01-20 10:13:40,965 - INFO - 📊 Processing: Pool eRZ58, GEX eRZ58:eRZ58_GEX60C, Guide eRZ58:eRZ58_gRNA60C
2026-01-20 10:13:40,982 - INFO - ⚠️  NOTE: args.skip_filtering is True. Original config min_umi (10) will be IGNORED.
2026-01-20 10:13:41,056 - INFO - 📋 Loading sample information...
2026-01-20 10:13:41,288 - INFO -    Sample: eRZ58:eRZ58_GEX60C
2026-01-20 10:13:41,289 - INFO -    Expected cells: 5000
2026-01-20 10:13:41,289 - INFO - 
📥 1. LOADING GEX DATA...
2026-01-20 10:13:41,290 - INFO -    Loading from: /scratch/users/shshanhe/eRZ58_default_251225/eRZ58_60C_analysis/eRZ58:eRZ58_GEX60C/kb_all_main_raw/counts_unfiltered_modified/adata.h5ad
2026-01-20 10:13:43,255 - INFO -    Loaded raw matrix: 453765 cells x 34183 genes
2026-01-20 10:13:43,346 - INFO -    Summed mature + nascent + ambiguous layers
2026-01-20 10:13:43,347 - INFO -    adata.X sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 10:13:43,362 - INFO -    Removing 425,990 barcodes with < 10 UMIs (93.9%)
2026-01-20 10:13:43,407 - INFO -    Kept 27,775 barcodes with >= 10 UMIs
2026-01-20 10:13:43,408 - INFO - 📊 Preparing GEX sample eRZ58:eRZ58_GEX60C (NO FILTERING)
2026-01-20 10:13:43,408 - INFO -    Keeping all 27,775 barcodes
2026-01-20 10:13:43,706 - INFO - 
📥 2. LOADING GUIDE DATA...
2026-01-20 10:13:43,799 - INFO - 📊 Preparing guide sample eRZ58:eRZ58_gRNA60C (NO FILTERING)
2026-01-20 10:13:43,801 - INFO -    Total guide barcodes: 76,872
2026-01-20 10:13:43,801 - INFO -    Using 27775 barcodes from paired GEX
2026-01-20 10:13:43,831 - INFO -    Found 11126 common barcodes between GEX and guide
2026-01-20 10:13:43,883 - INFO -    Filtered: 11126 cells x 1766 guides
2026-01-20 10:13:44,163 - INFO - 
🔗 3. COMBINING DATA...
2026-01-20 10:13:44,164 - INFO - 🧬 Adding guide data to GEX dataset...
2026-01-20 10:13:44,179 - INFO - 📊 Found 11126 common cells between GEX dataset and guide dataset
2026-01-20 10:13:44,179 - INFO - 📊 GEX dataset total cells: 27775
2026-01-20 10:13:44,180 - INFO - 📊 guide dataset total cells: 11126
2026-01-20 10:13:44,195 - INFO - 📊 Keeping ALL 27775 cells from GEX dataset (authoritative)
2026-01-20 10:13:44,195 - INFO - 📊 Adding data for 11126 matching cells from guide dataset
2026-01-20 10:13:44,197 - INFO - 🔄 Creating guide matrix for ALL GEX cells (GEX is authoritative)...
2026-01-20 10:13:44,206 - INFO - 📊 GEX cells (authoritative): 27,775
2026-01-20 10:13:44,207 - INFO - 📊 Guide cells available: 11,126
2026-01-20 10:13:44,207 - INFO - 📊 Cells with guide data: 11,126
2026-01-20 10:13:44,207 - INFO - 📊 Cells missing guide data: 16,649
2026-01-20 10:13:44,218 - INFO - 📊 Creating guide matrix for 27775 GEX cells (11126 with data)
2026-01-20 10:13:44,220 - INFO - ✅ Added guide_counts matrix (zeros for 16649 cells)
2026-01-20 10:13:44,221 - INFO - 📊 Added guide data: 1766 guides for 11126/27775 GEX cells
2026-01-20 10:13:44,222 - INFO - 📋 Guide data stored in obsm['guide_counts'] (zeros for cells without guide data)
2026-01-20 10:13:44,223 - INFO - 📊 Calculating guide statistics...
2026-01-20 10:13:44,224 - INFO -    Processing sparse guide matrix...
2026-01-20 10:13:44,225 - INFO -    guide_binary sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 10:13:44,228 - INFO - ✅ Guide statistics calculated: 27775 cells
2026-01-20 10:13:44,228 - INFO -    Mean guides per cell: 1.65
2026-01-20 10:13:44,229 - INFO -    % cells with guides: 40.1%
2026-01-20 10:13:44,367 - INFO - 
🏷️  4. ANNOTATING...
2026-01-20 10:13:44,369 - INFO - 🧬 Adding comprehensive gene annotations...
2026-01-20 10:13:44,371 - INFO -   📋 Loaded 182 ribosomal gene
2026-01-20 10:13:44,371 - INFO -   📋 Sample ribosomal gene: ['Rpl27a', 'Mrps36', 'Rpl31', 'Mrpl50', 'Mrpl28']
2026-01-20 10:13:44,400 - INFO -   📊 Calculating % cells expressed...
2026-01-20 10:13:44,420 - INFO -   🗃️  Loading gene annotations from gencode database...
2026-01-20 10:13:44,421 - INFO -   🔍 Sample gene IDs from adata.var.index: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
2026-01-20 10:13:44,422 - INFO -   🔍 Trying to annotate gene 1: ENSMUSG00000086053.2
2026-01-20 10:13:44,425 - INFO -     ✅ Found gene ENSMUSG00000086053.2 in database
2026-01-20 10:13:44,426 - INFO -     📋 Gene info: type=lncRNA, name=Gm15178, chr=1
2026-01-20 10:13:44,427 - INFO -   🔍 Trying to annotate gene 2: ENSMUSG00000100764.2
2026-01-20 10:13:44,431 - INFO -     ✅ Found gene ENSMUSG00000100764.2 in database
2026-01-20 10:13:44,432 - INFO -     📋 Gene info: type=lncRNA, name=Gm29155, chr=1
2026-01-20 10:13:44,433 - INFO -   🔍 Trying to annotate gene 3: ENSMUSG00000102095.2
2026-01-20 10:13:44,438 - INFO -     ✅ Found gene ENSMUSG00000102095.2 in database
2026-01-20 10:13:44,438 - INFO -     📋 Gene info: type=lncRNA, name=C730036E19Rik, chr=1
2026-01-20 10:13:44,440 - INFO -   🔍 Trying to annotate gene 4: ENSMUSG00000100635.2
2026-01-20 10:13:44,445 - INFO -     ✅ Found gene ENSMUSG00000100635.2 in database
2026-01-20 10:13:44,446 - INFO -     📋 Gene info: type=lncRNA, name=Gm29157, chr=1
2026-01-20 10:13:44,447 - INFO -   🔍 Trying to annotate gene 5: ENSMUSG00000100480.2
2026-01-20 10:13:44,451 - INFO -     ✅ Found gene ENSMUSG00000100480.2 in database
2026-01-20 10:13:44,453 - INFO -     📋 Gene info: type=lncRNA, name=Gm29156, chr=1
2026-01-20 10:16:57,453 - INFO -   📊 Annotation results: 34183 success, 0 missing, 0 errors
2026-01-20 10:16:57,454 - INFO -   🧬 Calculating mitochondrial gene scores...
2026-01-20 10:16:57,464 - INFO -   📊 Found 13 mitochondrial genes
2026-01-20 10:16:57,465 - INFO -   ✅ Added MT percentage and score using 13 genes
2026-01-20 10:16:57,466 - INFO -   🧬 Calculating ribosomal gene scores...
2026-01-20 10:16:57,467 - INFO -   📊 Found 176 ribosomal genes
2026-01-20 10:16:57,481 - INFO -   ✅ Added ribosomal percentage and score using 176 genes
2026-01-20 10:16:57,482 - INFO -   ✅ Annotated 34183/34183 genes
2026-01-20 10:16:57,524 - INFO -   📊 Gene types: {'protein_coding': 21959, 'lncRNA': 11527, 'IG_V_gene': 218, 'IG_V_pseudogene': 158, 'TR_V_gene': 144}
2026-01-20 10:16:57,526 - INFO -   📊 Ribosomal: 176
2026-01-20 10:16:57,527 - INFO -   📊 Cell cycle: 97
2026-01-20 10:16:57,545 - INFO -   📊 Chromosomes: 35 unique
2026-01-20 10:16:57,547 - INFO -   📊 Mean % cells expressed: 0.4%
2026-01-20 10:16:57,555 - INFO - 🧬 Adding mitochondrial gene metrics...
2026-01-20 10:16:57,566 - INFO - 📊 Using gene symbols: found 13 MT genes
2026-01-20 10:16:57,636 - INFO - 📊 Added mitochondrial metrics:
2026-01-20 10:16:57,637 - INFO -     Mean MT%: 5.26%
2026-01-20 10:16:57,638 - INFO -     Median MT%: 1.33%
2026-01-20 10:16:57,640 - INFO -    Mapping plate: plate1
2026-01-20 10:16:57,641 - INFO - 🗺️ Mapping cells to biological samples using plate plate1...
2026-01-20 10:16:57,780 - INFO - 🔍 Extracting Round1 barcodes from cell names...
2026-01-20 10:16:57,792 - INFO - 📊 Extracted barcodes for 27775 cells
2026-01-20 10:16:57,817 - INFO - 📊 27775/27775 cells have valid Round1 barcodes
2026-01-20 10:16:57,818 - INFO - ✅ All Round1 barcodes are valid
2026-01-20 10:16:59,441 - INFO - 📊 Mapped 27775/27775 cells to biological samples
2026-01-20 10:16:59,445 - INFO - 
💾 5. SAVING OUTPUTS...
2026-01-20 10:17:04,068 - INFO -    Saved MTX files to /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX60C_unfilered
2026-01-20 10:17:04,069 - INFO - 
✅ Ready for Cell Calling Benchmark!
2026-01-20 10:17:04,069 - INFO -    Output saved to: /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX60C_unfilered2026-01-20 09:41:01,263 - INFO - 
================================================================================
2026-01-20 09:41:01,263 - INFO - 🚀 STARTING SUBLIBRARY FILTERING AND ANNOTATION PIPELINE (NOTEBOOK MODE)
2026-01-20 09:41:01,264 - INFO - 📊 Processing: Pool eRZ58, GEX eRZ58:eRZ58_GEX40B, Guide eRZ58:eRZ58_gRNA40B
2026-01-20 09:41:01,280 - INFO - ⚠️  NOTE: args.skip_filtering is True. Original config min_umi (10) will be IGNORED.
2026-01-20 09:41:01,284 - INFO - 📋 Loading sample information...
2026-01-20 09:41:01,305 - INFO -    Sample: eRZ58:eRZ58_GEX40B
2026-01-20 09:41:01,305 - INFO -    Expected cells: 5000
2026-01-20 09:41:01,306 - INFO - 
📥 1. LOADING GEX DATA...
2026-01-20 09:41:01,307 - INFO -    Loading from: /scratch/users/shshanhe/eRZ58_default_251225/eRZ58_40B_analysis/eRZ58:eRZ58_GEX40B/kb_all_main_raw/counts_unfiltered_modified/adata.h5ad
2026-01-20 09:41:06,467 - INFO -    Loaded raw matrix: 406853 cells x 34183 genes
2026-01-20 09:41:06,666 - INFO -    Summed mature + nascent + ambiguous layers
2026-01-20 09:41:06,667 - INFO -    adata.X sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 09:41:06,686 - INFO -    Removing 386,270 barcodes with < 10 UMIs (94.9%)
2026-01-20 09:41:06,770 - INFO -    Kept 20,583 barcodes with >= 10 UMIs
2026-01-20 09:41:06,771 - INFO - 📊 Preparing GEX sample eRZ58:eRZ58_GEX40B (NO FILTERING)
2026-01-20 09:41:06,772 - INFO -    Keeping all 20,583 barcodes
2026-01-20 09:41:07,633 - INFO - 
📥 2. LOADING GUIDE DATA...
2026-01-20 09:41:07,720 - INFO - 📊 Preparing guide sample eRZ58:eRZ58_gRNA40B (NO FILTERING)
2026-01-20 09:41:07,721 - INFO -    Total guide barcodes: 80,613
2026-01-20 09:41:07,722 - INFO -    Using 20583 barcodes from paired GEX
2026-01-20 09:41:07,741 - INFO -    Found 9176 common barcodes between GEX and guide
2026-01-20 09:41:07,750 - INFO -    Filtered: 9176 cells x 1766 guides
2026-01-20 09:41:08,067 - INFO - 
🔗 3. COMBINING DATA...
2026-01-20 09:41:08,068 - INFO - 🧬 Adding guide data to GEX dataset...
2026-01-20 09:41:08,492 - INFO - 📊 Found 9176 common cells between GEX dataset and guide dataset
2026-01-20 09:41:08,493 - INFO - 📊 GEX dataset total cells: 20583
2026-01-20 09:41:08,493 - INFO - 📊 guide dataset total cells: 9176
2026-01-20 09:41:08,507 - INFO - 📊 Keeping ALL 20583 cells from GEX dataset (authoritative)
2026-01-20 09:41:08,508 - INFO - 📊 Adding data for 9176 matching cells from guide dataset
2026-01-20 09:41:08,509 - INFO - 🔄 Creating guide matrix for ALL GEX cells (GEX is authoritative)...
2026-01-20 09:41:08,518 - INFO - 📊 GEX cells (authoritative): 20,583
2026-01-20 09:41:08,519 - INFO - 📊 Guide cells available: 9,176
2026-01-20 09:41:08,520 - INFO - 📊 Cells with guide data: 9,176
2026-01-20 09:41:08,520 - INFO - 📊 Cells missing guide data: 11,407
2026-01-20 09:41:08,528 - INFO - 📊 Creating guide matrix for 20583 GEX cells (9176 with data)
2026-01-20 09:41:08,530 - INFO - ✅ Added guide_counts matrix (zeros for 11407 cells)
2026-01-20 09:41:08,531 - INFO - 📊 Added guide data: 1766 guides for 9176/20583 GEX cells
2026-01-20 09:41:08,532 - INFO - 📋 Guide data stored in obsm['guide_counts'] (zeros for cells without guide data)
2026-01-20 09:41:08,533 - INFO - 📊 Calculating guide statistics...
2026-01-20 09:41:08,534 - INFO -    Processing sparse guide matrix...
2026-01-20 09:41:08,535 - INFO -    guide_binary sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 09:41:08,547 - INFO - ✅ Guide statistics calculated: 20583 cells
2026-01-20 09:41:08,549 - INFO -    Mean guides per cell: 2.10
2026-01-20 09:41:08,550 - INFO -    % cells with guides: 44.6%
2026-01-20 09:41:08,697 - INFO - 
🏷️  4. ANNOTATING...
2026-01-20 09:41:08,698 - INFO - 🧬 Adding comprehensive gene annotations...
2026-01-20 09:41:08,718 - INFO -   📋 Loaded 182 ribosomal gene
2026-01-20 09:41:08,720 - INFO -   📋 Sample ribosomal gene: ['Rpl27a', 'Mrps36', 'Rpl31', 'Mrpl50', 'Mrpl28']
2026-01-20 09:41:08,892 - INFO -   📊 Calculating % cells expressed...
2026-01-20 09:41:10,681 - INFO -   🗃️  Loading gene annotations from gencode database...
2026-01-20 09:41:10,702 - INFO -   🔍 Sample gene IDs from adata.var.index: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
2026-01-20 09:41:10,704 - INFO -   🔍 Trying to annotate gene 1: ENSMUSG00000086053.2
2026-01-20 09:41:10,793 - INFO -     ✅ Found gene ENSMUSG00000086053.2 in database
2026-01-20 09:41:10,793 - INFO -     📋 Gene info: type=lncRNA, name=Gm15178, chr=1
2026-01-20 09:41:10,795 - INFO -   🔍 Trying to annotate gene 2: ENSMUSG00000100764.2
2026-01-20 09:41:10,847 - INFO -     ✅ Found gene ENSMUSG00000100764.2 in database
2026-01-20 09:41:10,848 - INFO -     📋 Gene info: type=lncRNA, name=Gm29155, chr=1
2026-01-20 09:41:10,850 - INFO -   🔍 Trying to annotate gene 3: ENSMUSG00000102095.2
2026-01-20 09:41:10,864 - INFO -     ✅ Found gene ENSMUSG00000102095.2 in database
2026-01-20 09:41:10,864 - INFO -     📋 Gene info: type=lncRNA, name=C730036E19Rik, chr=1
2026-01-20 09:41:10,866 - INFO -   🔍 Trying to annotate gene 4: ENSMUSG00000100635.2
2026-01-20 09:41:10,903 - INFO -     ✅ Found gene ENSMUSG00000100635.2 in database
2026-01-20 09:41:10,904 - INFO -     📋 Gene info: type=lncRNA, name=Gm29157, chr=1
2026-01-20 09:41:10,906 - INFO -   🔍 Trying to annotate gene 5: ENSMUSG00000100480.2
2026-01-20 09:41:10,924 - INFO -     ✅ Found gene ENSMUSG00000100480.2 in database
2026-01-20 09:41:10,925 - INFO -     📋 Gene info: type=lncRNA, name=Gm29156, chr=1
2026-01-20 09:55:19,045 - INFO -   📊 Annotation results: 34183 success, 0 missing, 0 errors
2026-01-20 09:55:19,046 - INFO -   🧬 Calculating mitochondrial gene scores...
2026-01-20 09:55:19,097 - INFO -   📊 Found 13 mitochondrial genes
2026-01-20 09:55:19,099 - INFO -   ✅ Added MT percentage and score using 13 genes
2026-01-20 09:55:19,099 - INFO -   🧬 Calculating ribosomal gene scores...
2026-01-20 09:55:19,101 - INFO -   📊 Found 176 ribosomal genes
2026-01-20 09:55:19,349 - INFO -   ✅ Added ribosomal percentage and score using 176 genes
2026-01-20 09:55:19,349 - INFO -   ✅ Annotated 34183/34183 genes
2026-01-20 09:55:19,355 - INFO -   📊 Gene types: {'protein_coding': 21959, 'lncRNA': 11527, 'IG_V_gene': 218, 'IG_V_pseudogene': 158, 'TR_V_gene': 144}
2026-01-20 09:55:19,355 - INFO -   📊 Ribosomal: 176
2026-01-20 09:55:19,356 - INFO -   📊 Cell cycle: 97
2026-01-20 09:55:19,358 - INFO -   📊 Chromosomes: 35 unique
2026-01-20 09:55:19,360 - INFO -   📊 Mean % cells expressed: 0.5%
2026-01-20 09:55:19,366 - INFO - 🧬 Adding mitochondrial gene metrics...
2026-01-20 09:55:19,378 - INFO - 📊 Using gene symbols: found 13 MT genes
2026-01-20 09:55:19,601 - INFO - 📊 Added mitochondrial metrics:
2026-01-20 09:55:19,603 - INFO -     Mean MT%: 5.75%
2026-01-20 09:55:19,604 - INFO -     Median MT%: 2.24%
2026-01-20 09:55:19,606 - INFO -    Mapping plate: plate1
2026-01-20 09:55:19,606 - INFO - 🗺️ Mapping cells to biological samples using plate plate1...
2026-01-20 09:55:19,670 - INFO - 🔍 Extracting Round1 barcodes from cell names...
2026-01-20 09:55:19,678 - INFO - 📊 Extracted barcodes for 20583 cells
2026-01-20 09:55:19,680 - INFO - 📊 20583/20583 cells have valid Round1 barcodes
2026-01-20 09:55:19,681 - INFO - ✅ All Round1 barcodes are valid
2026-01-20 09:55:20,916 - INFO - 📊 Mapped 20583/20583 cells to biological samples
2026-01-20 09:55:20,918 - INFO - 
💾 5. SAVING OUTPUTS...
2026-01-20 09:55:25,522 - INFO -    Saved MTX files to /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX40B_unfilered
2026-01-20 09:55:25,522 - INFO - 
✅ Ready for Cell Calling Benchmark!
2026-01-20 09:55:25,524 - INFO -    Output saved to: /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX40B_unfilered

2026-01-20 08:50:33,840 - INFO - 
================================================================================
2026-01-20 08:50:33,841 - INFO - 🚀 STARTING SUBLIBRARY FILTERING AND ANNOTATION PIPELINE (NOTEBOOK MODE)
2026-01-20 08:50:33,842 - INFO - 📊 Processing: Pool eRZ58, GEX eRZ58:eRZ58_GEX20A, Guide eRZ58:eRZ58_gRNA20A
2026-01-20 08:50:33,860 - INFO - ⚠️  NOTE: args.skip_filtering is True. Original config min_umi (10) will be IGNORED.
2026-01-20 08:50:33,864 - INFO - 📋 Loading sample information...
2026-01-20 08:50:33,880 - INFO -    Sample: eRZ58:eRZ58_GEX20A
2026-01-20 08:50:33,881 - INFO -    Expected cells: 5000
2026-01-20 08:50:33,882 - INFO - 
📥 1. LOADING GEX DATA...
2026-01-20 08:50:33,882 - INFO -    Loading from: /scratch/users/shshanhe/eRZ58_default_251225/eRZ58_20A_analysis/eRZ58:eRZ58_GEX20A/kb_all_main_raw/counts_unfiltered_modified/adata.h5ad
2026-01-20 08:50:34,254 - INFO -    Loaded raw matrix: 400356 cells x 34183 genes
2026-01-20 08:50:34,334 - INFO -    Summed mature + nascent + ambiguous layers
2026-01-20 08:50:34,335 - INFO -    adata.X sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 08:50:34,348 - INFO -    Removing 385,698 barcodes with < 10 UMIs (96.3%)
2026-01-20 08:50:34,378 - INFO -    Kept 14,658 barcodes with >= 10 UMIs
2026-01-20 08:50:34,379 - INFO - 📊 Preparing GEX sample eRZ58:eRZ58_GEX20A (NO FILTERING)
2026-01-20 08:50:34,379 - INFO -    Keeping all 14,658 barcodes
2026-01-20 08:50:34,536 - INFO - 
📥 2. LOADING GUIDE DATA...
2026-01-20 08:50:34,618 - INFO - 📊 Preparing guide sample eRZ58:eRZ58_gRNA20A (NO FILTERING)
2026-01-20 08:50:34,619 - INFO -    Total guide barcodes: 72,518
2026-01-20 08:50:34,619 - INFO -    Using 14658 barcodes from paired GEX
2026-01-20 08:50:34,666 - INFO -    Found 7409 common barcodes between GEX and guide
2026-01-20 08:50:34,819 - INFO -    Filtered: 7409 cells x 1766 guides
2026-01-20 08:50:35,097 - INFO - 
🔗 3. COMBINING DATA...
2026-01-20 08:50:35,098 - INFO - 🧬 Adding guide data to GEX dataset...
2026-01-20 08:50:35,371 - INFO - 📊 Found 7409 common cells between GEX dataset and guide dataset
2026-01-20 08:50:35,372 - INFO - 📊 GEX dataset total cells: 14658
2026-01-20 08:50:35,373 - INFO - 📊 guide dataset total cells: 7409
2026-01-20 08:50:35,400 - INFO - 📊 Keeping ALL 14658 cells from GEX dataset (authoritative)
2026-01-20 08:50:35,401 - INFO - 📊 Adding data for 7409 matching cells from guide dataset
2026-01-20 08:50:35,403 - INFO - 🔄 Creating guide matrix for ALL GEX cells (GEX is authoritative)...
2026-01-20 08:50:35,407 - INFO - 📊 GEX cells (authoritative): 14,658
2026-01-20 08:50:35,408 - INFO - 📊 Guide cells available: 7,409
2026-01-20 08:50:35,408 - INFO - 📊 Cells with guide data: 7,409
2026-01-20 08:50:35,409 - INFO - 📊 Cells missing guide data: 7,249
2026-01-20 08:50:35,413 - INFO - 📊 Creating guide matrix for 14658 GEX cells (7409 with data)
2026-01-20 08:50:35,586 - INFO - ✅ Added guide_counts matrix (zeros for 7249 cells)
2026-01-20 08:50:35,587 - INFO - 📊 Added guide data: 1766 guides for 7409/14658 GEX cells
2026-01-20 08:50:35,587 - INFO - 📋 Guide data stored in obsm['guide_counts'] (zeros for cells without guide data)
2026-01-20 08:50:35,589 - INFO - 📊 Calculating guide statistics...
2026-01-20 08:50:35,590 - INFO -    Processing sparse guide matrix...
2026-01-20 08:50:35,618 - INFO -    guide_binary sparsity: True, type: <class 'scipy.sparse._csr.csr_matrix'>
2026-01-20 08:50:35,687 - INFO - ✅ Guide statistics calculated: 14658 cells
2026-01-20 08:50:35,774 - INFO -    Mean guides per cell: 2.83
2026-01-20 08:50:35,827 - INFO -    % cells with guides: 50.5%
2026-01-20 08:50:35,970 - INFO - 
🏷️  4. ANNOTATING...
2026-01-20 08:50:35,971 - INFO - 🧬 Adding comprehensive gene annotations...
2026-01-20 08:50:36,021 - INFO -   📋 Loaded 182 ribosomal gene
2026-01-20 08:50:36,022 - INFO -   📋 Sample ribosomal gene: ['Rpl27a', 'Mrps36', 'Rpl31', 'Mrpl50', 'Mrpl28']
2026-01-20 08:50:36,252 - INFO -   📊 Calculating % cells expressed...
2026-01-20 08:50:39,907 - INFO -   🗃️  Loading gene annotations from gencode database...
2026-01-20 08:50:39,921 - INFO -   🔍 Sample gene IDs from adata.var.index: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
2026-01-20 08:50:39,922 - INFO -   🔍 Trying to annotate gene 1: ENSMUSG00000086053.2
2026-01-20 08:50:39,995 - INFO -     ✅ Found gene ENSMUSG00000086053.2 in database
2026-01-20 08:50:39,996 - INFO -     📋 Gene info: type=lncRNA, name=Gm15178, chr=1
2026-01-20 08:50:39,997 - INFO -   🔍 Trying to annotate gene 2: ENSMUSG00000100764.2
2026-01-20 08:50:40,053 - INFO -     ✅ Found gene ENSMUSG00000100764.2 in database
2026-01-20 08:50:40,054 - INFO -     📋 Gene info: type=lncRNA, name=Gm29155, chr=1
2026-01-20 08:50:40,055 - INFO -   🔍 Trying to annotate gene 3: ENSMUSG00000102095.2
2026-01-20 08:50:40,071 - INFO -     ✅ Found gene ENSMUSG00000102095.2 in database
2026-01-20 08:50:40,071 - INFO -     📋 Gene info: type=lncRNA, name=C730036E19Rik, chr=1
2026-01-20 08:50:40,072 - INFO -   🔍 Trying to annotate gene 4: ENSMUSG00000100635.2
2026-01-20 08:50:40,136 - INFO -     ✅ Found gene ENSMUSG00000100635.2 in database
2026-01-20 08:50:40,137 - INFO -     📋 Gene info: type=lncRNA, name=Gm29157, chr=1
2026-01-20 08:50:40,138 - INFO -   🔍 Trying to annotate gene 5: ENSMUSG00000100480.2
2026-01-20 08:50:40,197 - INFO -     ✅ Found gene ENSMUSG00000100480.2 in database
2026-01-20 08:50:40,197 - INFO -     📋 Gene info: type=lncRNA, name=Gm29156, chr=1
2026-01-20 09:04:35,884 - INFO -   📊 Annotation results: 34183 success, 0 missing, 0 errors
2026-01-20 09:04:35,885 - INFO -   🧬 Calculating mitochondrial gene scores...
2026-01-20 09:04:35,947 - INFO -   📊 Found 13 mitochondrial genes
2026-01-20 09:04:35,948 - INFO -   ✅ Added MT percentage and score using 13 genes
2026-01-20 09:04:35,949 - INFO -   🧬 Calculating ribosomal gene scores...
2026-01-20 09:04:35,975 - INFO -   📊 Found 176 ribosomal genes
2026-01-20 09:04:37,150 - INFO -   ✅ Added ribosomal percentage and score using 176 genes
2026-01-20 09:04:37,153 - INFO -   ✅ Annotated 34183/34183 genes
2026-01-20 09:04:37,317 - INFO -   📊 Gene types: {'protein_coding': 21959, 'lncRNA': 11527, 'IG_V_gene': 218, 'IG_V_pseudogene': 158, 'TR_V_gene': 144}
2026-01-20 09:04:37,319 - INFO -   📊 Ribosomal: 176
2026-01-20 09:04:37,320 - INFO -   📊 Cell cycle: 97
2026-01-20 09:04:37,335 - INFO -   📊 Chromosomes: 35 unique
2026-01-20 09:04:37,336 - INFO -   📊 Mean % cells expressed: 0.6%
2026-01-20 09:04:37,340 - INFO - 🧬 Adding mitochondrial gene metrics...
2026-01-20 09:04:37,349 - INFO - 📊 Using gene symbols: found 13 MT genes
2026-01-20 09:04:37,588 - INFO - 📊 Added mitochondrial metrics:
2026-01-20 09:04:37,589 - INFO -     Mean MT%: 7.33%
2026-01-20 09:04:37,841 - INFO -     Median MT%: 4.17%
2026-01-20 09:04:37,843 - INFO -    Mapping plate: plate1
2026-01-20 09:04:37,844 - INFO - 🗺️ Mapping cells to biological samples using plate plate1...
2026-01-20 09:04:37,929 - INFO - 🔍 Extracting Round1 barcodes from cell names...
2026-01-20 09:04:37,939 - INFO - 📊 Extracted barcodes for 14658 cells
2026-01-20 09:04:37,971 - INFO - 📊 14658/14658 cells have valid Round1 barcodes
2026-01-20 09:04:37,972 - INFO - ✅ All Round1 barcodes are valid
2026-01-20 09:04:38,975 - INFO - 📊 Mapped 14658/14658 cells to biological samples
2026-01-20 09:04:38,977 - INFO - 
💾 5. SAVING OUTPUTS...
2026-01-20 09:04:46,125 - INFO -    Saved MTX files to /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX20A_unfilered
2026-01-20 09:04:46,127 - INFO - 
✅ Ready for Cell Calling Benchmark!
2026-01-20 09:04:46,128 - INFO -    Output saved to: /oak/stanford/groups/engreitz/Users/shshanhe/project/perturb_pipeline/analysis/dev_cell_calling_eRZ58:eRZ58_GEX20A_unfilered


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2578487800.py, line 1)